Harris Corner Detection

In [1]:
import cv2 
import numpy as np

In [4]:
img = cv2.imread('building.jpg', 1)
h, w, ch = img.shape
print([h, w, ch])

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imshow('building_grayscale', gray)
cv2.waitKey()
cv2.destroyAllWindows()

[640, 480, 3]


Global variable

In [6]:
# window size: 7x7
wsize = 7

# Apply Gaussian Blur
gray = cv2.GaussianBlur(gray, (0, 0), 0.5)
cv2.imshow('building_blur', gray)
cv2.waitKey()
cv2.destroyAllWindows()

Get Harris Corner Map

In [7]:
def getCornerMap(gray, window_size, lamda=0.04):
  gray = gray.astype(np.float32) / 255.0
  h, w = gray.shape

  Iy, Ix = np.gradient(gray)

  Ix2 = Ix ** 2
  Ixy = Ix * Iy
  Iy2 = Iy ** 2

  sum_dx2 = np.zeros_like(gray)
  sum_dy2 = np.zeros_like(gray)
  sum_dxy = np.zeros_like(gray)

  for i in range(window_size):
      for j in range(window_size):
          sum_dx2 += np.roll(np.roll(Ix2, -i, axis=0), -j, axis=1)
          sum_dy2 += np.roll(np.roll(Iy2, -i, axis=0), -j, axis=1)
          sum_dxy += np.roll(np.roll(Ixy, -i, axis=0), -j, axis=1)

  det_M = (sum_dx2 * sum_dy2) - (sum_dxy * sum_dxy)
  trace_M = sum_dx2 + sum_dy2
  hmap = det_M - lamda * (trace_M ** 2)

  return hmap

In [9]:
hmap = getCornerMap(gray, wsize, 0.04)
print(np.max(hmap))

cv2.imshow('hmap', hmap*512)
cv2.waitKey()
cv2.destroyAllWindows()

1.1452491


In [10]:
img_corner = img.copy()
nms = 5
max_corner = np.max(hmap)

for v in range(nms, h-nms):
  for u in range(nms, w-nms):
    if hmap[v,u] >= np.max(hmap[v-nms:v+nms, u-nms:u+nms]) and hmap[v,u] > 0.1 * max_corner:
      cv2.circle(img_corner, (u, v), 3, (0, 255, 255), -1)
      hmap[h-nms:h+nms, w-nms:w+nms] = 0

cv2.imshow('img_corner', img_corner)
cv2.waitKey()
cv2.destroyAllWindows()